In [1]:
import os
import sys

In [2]:
import numpy
import pandas

In [3]:
import spacy

In [4]:
import  contractions

In [5]:
import spellchecker

In [6]:
import missingno

In [86]:
import sklearn

In [231]:
import sklearn.cluster
import sklearn.metrics
import sklearn.ensemble
import sklearn.model_selection

In [190]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import FeatureAgglomeration

In [216]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [237]:
from sklearn.model_selection import train_test_split

In [238]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

In [7]:
crisis_logger = pandas.read_csv("../../Data/CrisisLogger/crisislogger.csv")

In [8]:
crisis_pa_adlt_202004 = pandas.read_csv("../../Data/ProlificAcademic/April 2020/Data/CRISIS_Adult_April_2020.csv")
crisis_pa_prnt_202004 = pandas.read_csv("../../Data/ProlificAcademic/April 2020/Data/CRISIS_Parent_April_2020.csv")
crisis_pa_adlt_202004["ind_adlt"] = 1
crisis_pa_adlt_202004["ind_prnt"] = 0
crisis_pa_adlt_202004["num_yr"] = 2020
crisis_pa_adlt_202004["num_mnth"] = 4
crisis_pa_prnt_202004["ind_adlt"] = 0
crisis_pa_prnt_202004["ind_prnt"] = 1
crisis_pa_prnt_202004["num_yr"] = 2020
crisis_pa_prnt_202004["num_mnth"] = 4

In [9]:
crisis_pa_adlt_202005 = pandas.read_csv("../../Data/ProlificAcademic/May 2020/Data/CRISIS_Adult_May_2020.csv")
crisis_pa_prnt_202005 = pandas.read_csv("../../Data/ProlificAcademic/May 2020/Data/CRISIS_Parent_May_2020.csv")
crisis_pa_adlt_202005["ind_adlt"] = 1
crisis_pa_adlt_202005["ind_prnt"] = 0
crisis_pa_adlt_202005["num_yr"] = 2020
crisis_pa_adlt_202005["num_mnth"] = 5
crisis_pa_prnt_202005["ind_adlt"] = 0
crisis_pa_prnt_202005["ind_prnt"] = 1
crisis_pa_prnt_202005["num_yr"] = 2020
crisis_pa_prnt_202005["num_mnth"] = 5

In [10]:
crisis_pa_adlt_202011 = pandas.read_csv("../../Data/ProlificAcademic/November 2020/Data/CRISIS_Adult_November_2020.csv", low_memory=False)
crisis_pa_prnt_202011 = pandas.read_csv("../../Data/ProlificAcademic/November 2020/Data/CRISIS_Parent_November_2020.csv", low_memory=False)
crisis_pa_adlt_202011["ind_adlt"] = 1
crisis_pa_adlt_202011["ind_prnt"] = 0
crisis_pa_adlt_202011["num_yr"] = 2020
crisis_pa_adlt_202011["num_mnth"] = 11
crisis_pa_prnt_202011["ind_adlt"] = 0
crisis_pa_prnt_202011["ind_prnt"] = 1
crisis_pa_prnt_202011["num_yr"] = 2020
crisis_pa_prnt_202011["num_mnth"] = 11

In [11]:
crisis_pa_adlt_202104 = pandas.read_csv("../../Data/ProlificAcademic/April 2021/Data/CRISIS_Adult_April_2021.csv", low_memory=False)
crisis_pa_prnt_202104 = pandas.read_csv("../../Data/ProlificAcademic/April 2021/Data/CRISIS_Parent_April_2021.csv", low_memory=False)
crisis_pa_adlt_202104["ind_adlt"] = 1
crisis_pa_adlt_202104["ind_prnt"] = 0
crisis_pa_adlt_202104["num_yr"] = 2021
crisis_pa_adlt_202104["num_mnth"] = 4
crisis_pa_prnt_202104["ind_adlt"] = 0
crisis_pa_prnt_202104["ind_prnt"] = 1
crisis_pa_prnt_202104["num_yr"] = 2021
crisis_pa_prnt_202104["num_mnth"] = 4

In [12]:
crisis_pa_adlt = pandas.concat([crisis_pa_adlt_202004, crisis_pa_adlt_202005, crisis_pa_adlt_202011, crisis_pa_adlt_202104])

In [13]:
crisis_pa_prnt = pandas.concat([crisis_pa_prnt_202004, crisis_pa_prnt_202005, crisis_pa_prnt_202011, crisis_pa_prnt_202104])

In [14]:
# Data frame for Focus Group conversations
gaming_fg = pandas.read_csv("../data_preparation/gaming_group.csv")
social_fg = pandas.read_csv("../data_preparation/social_group.csv")
media_fg = pandas.read_csv("../data_preparation/media_group.csv")
lp_fg = pandas.read_csv("../data_preparation/lowpiu.csv")

In [15]:
nlp = spacy.load("en_core_web_lg")

In [16]:
all_text = list(crisis_logger.transcriptions)

In [17]:
all_text.extend(list(social_fg.parent_answer))
all_text.extend(list(gaming_fg.parent_answer))
all_text.extend(list(media_fg.parent_answer))
all_text.extend(list(lp_fg.parent_answer))

In [18]:
all_text.extend(crisis_pa_adlt["specifypositive"][~pandas.isnull(crisis_pa_adlt["specifypositive"])])

In [19]:
all_text.extend(crisis_pa_prnt["specifypositive"][~pandas.isnull(crisis_pa_prnt["specifypositive"])])

In [20]:
all_text_cfixed = [contractions.fix(text) for text in all_text]

In [21]:
crisis_pa_adlt.describe()

,country,age,sex,raceethnicity___1,raceethnicity___2,raceethnicity___3,raceethnicity___4,raceethnicity___5,raceethnicity___6,raceethnicity___7,...,iat_12,iat_13,iat_14,iat_15,iat_16,iat_17,iat_18,iat_19,iat_20,internet_addiction_test_iat_complete
count,10351.000000,10340.000000,2839.000000,5091.000000,5091.000000,5091.000000,5091.000000,5091.000000,5091.000000,5091.000000,...,2038.000000,2038.000000,2037.000000,2040.000000,2042.000000,2039.000000,2040.000000,2040.000000,2037.000000,2125.000000
mean,186.342189,38.956383,1.579429,0.677274,0.022392,0.020821,0.005303,0.017285,0.047338,0.103712,...,1.371443,0.792934,1.208640,0.726471,1.526934,1.299657,0.677941,0.912745,0.696613,1.923765
std,3.335535,14.746920,0.501526,0.467565,0.147971,0.142799,0.072639,0.130346,0.212383,0.304917,...,1.238758,0.858301,1.179043,0.904832,1.303337,1.160756,0.893387,1.082762,0.886071,0.383051
min,17.000000,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,186.000000,27.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,186.000000,36.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,2.000000
75%,187.000000,51.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000
max,187.000000,83.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,2.000000


In [22]:
# Initialize a dictionary to add data
missing_values_dict = {}
missing_values_dict["nme_col"] = []
missing_values_dict["cnt_missing"] = []
missing_values_dict["cnt_avl"] = []
# Print the percentage of non available values across each column.
for c in crisis_pa_adlt.columns:
    missing_values_dict["nme_col"].append(c)
    missing_values_dict["cnt_missing"].append(len(crisis_pa_adlt[pandas.isna(crisis_pa_adlt[c])]))
    missing_values_dict["cnt_avl"].append(len(crisis_pa_adlt[~pandas.isna(crisis_pa_adlt[c])]))
# Missing values dataframe.
crisis_pa_adlt_msng = pandas.DataFrame(missing_values_dict)

In [23]:
# Rationalize tall / height
crisis_pa_adlt[crisis_pa_adlt["measurementheight"]==1][["tall", "measurementheight"]]
# If height is in inches, convert it to cm
crisis_pa_adlt["tall_cm"] = crisis_pa_adlt["tall"]
crisis_pa_adlt["tall_cm"] = crisis_pa_adlt[["tall", "measurementheight"]].apply(lambda r: r[0] if r[1]==2 else r[0] * 2.54, axis=1)
# 10th and 95th percentile height
tall_cm_095 = crisis_pa_adlt["tall_cm"].quantile(0.95)
tall_cm_010 = crisis_pa_adlt["tall_cm"].quantile(0.10)
# Filter outliers
crisis_pa_adlt["tall_cm"] = crisis_pa_adlt["tall_cm"].apply(lambda r: r if (r>=tall_cm_010 and r<=tall_cm_095) else numpy.nan)

In [24]:
# Rationalize weight
crisis_pa_adlt[crisis_pa_adlt["measurementweight"]==1][["weight", "measurementweight"]]
# If weight is in lbs, convert it to kg
crisis_pa_adlt["weight_kg"] = crisis_pa_adlt["weight"]
crisis_pa_adlt["weight_kg"] = crisis_pa_adlt[["weight", "measurementweight"]].apply(lambda r: r[0] if r[1]==2 else r[0] * 0.453, axis=1)
# Percentiles
weight_kg_005 = crisis_pa_adlt["weight_kg"].quantile(0.05)
weight_kg_095 = crisis_pa_adlt["weight_kg"].quantile(0.95)
# Filter outliers
crisis_pa_adlt["weight_kg"] = crisis_pa_adlt["weight_kg"].apply(lambda r: r if (r>=weight_kg_005 and r<=weight_kg_095) else numpy.nan)

In [155]:
# Physical health encoding
pandas.get_dummies(crisis_pa_adlt["physicalhealth"], prefix="physicalhealth")

,physicalhealth_1.0,physicalhealth_2.0,physicalhealth_3.0,physicalhealth_4.0,physicalhealth_5.0
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
...,...,...,...,...,...
2120,0,0,0,0,0
2121,0,0,0,0,0
2122,0,0,0,0,0
2123,0,0,0,0,0


In [172]:
crisis_pa_adlt["hoursofsleepweekdays_2"]

0       3.0
1       3.0
2       2.0
3       2.0
4       2.0
       ... 
2120    NaN
2121    NaN
2122    NaN
2123    NaN
2124    NaN
Name: hoursofsleepweekdays_2, Length: 10364, dtype: float64

In [26]:
len(crisis_pa_adlt[crisis_pa_adlt["hoursofsleepweekdays"]==crisis_pa_adlt["hoursofsleepweekdays_2"]])

3507

In [68]:
len(crisis_pa_adlt[pandas.isna(crisis_pa_adlt["substance___3"])])

6016

In [64]:
crisis_pa_adlt["substancecomplaint"].describe()

count    2550.000000
mean        0.050196
std         0.218392
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: substancecomplaint, dtype: float64

In [252]:
# Columns for demographic clustering
dmg_hlth_cols = [
    "age",
    "gender",
    "tall_cm",
    "weight_kg",
    "cigarettesyn",
    "healthconditions___1",
    "healthconditions___2",
    "healthconditions___3",
    "healthconditions___4",
    "healthconditions___5",
    "healthconditions___6",
    "healthconditions___7",
    "healthconditions___8",
    "healthconditions___9",
    "healthconditions___10",
    "healthconditions___11",
    "healthconditions___12",
    "healthconditions___13",
    "healthconditions___14",
    "healthconditions___15",
    "healthconditions___16",
    "healthconditions___17",
    "healthconditions___18",
    "healthconditions___19",
    "healthconditions___20",
    "hoursofsleepweekdays",
    "hoursofsleepweekends",
    "troublesleeping",
    "sleepquality"
]


In [70]:
dmg_subs_cols = [
    "alcohol",
    "cigarettes",
    "vape",
    "tobacco",
    "marijuana",
    "heroin",
    "cocaine",
    "rxpain",
    "rxsleeping",
    "rxstimulants",
    "rxother",
    "substanceproblems",
    "substancecomplaint"
]

In [75]:
dmg_subs_cols_imp_1 = [
    "alcohol",
    "cigarettes",
    "vape",
    "tobacco",
    "marijuana",
    "heroin",
    "cocaine",
    "rxpain",
    "rxsleeping",
    "rxstimulants",
    "rxother"
] 

In [165]:
dmg_subs_cols_imp_1_sub = [
    "alcohol",
    "cigarettes",
    "vape",
    "tobacco",
    "marijuana",
    "heroin",
    "cocaine"
]
dmg_subs_cols_imp_1_med = [
    "rxpain",
    "rxsleeping",
    "rxstimulants",
    "rxother"
]

In [74]:
# Creating a dictionary to map substance use to frequency
# 1, Not at all | 2, Rarely | 3, Once a month | 4, Several times a month | 5, Once a week | 6, Several times a week | 7, Once a day | 8, More than once a day
subs_freq_dict = {
    1: 0,
    2: 0.01,
    3: 0.03,
    4: 0.075,
    5: 0.15,
    6: 0.33,
    7: 1,
    8: 3
}

In [82]:
for col in dmg_subs_cols_imp_1:
    # First apply 1 in cases where response isn't provided
    crisis_pa_adlt[col + "_imp"] = crisis_pa_adlt[col].apply(lambda r: 1.0 if numpy.isnan(r) else r)
    crisis_pa_adlt[col + "_imp"] = crisis_pa_adlt[col + "_imp"].astype(int)
    crisis_pa_adlt[col + "_imp_freq"] = crisis_pa_adlt[col + "_imp"].apply(lambda r: subs_freq_dict[r])

In [90]:
# Other columns in substances
crisis_pa_adlt["substanceproblems_imp"] = crisis_pa_adlt["substanceproblems"].apply(lambda r: 0 if numpy.isnan(r) else r).astype(int)
crisis_pa_adlt["substancecomplaint_imp"] = crisis_pa_adlt["substancecomplaint"].apply(lambda r: 0 if numpy.isnan(r) else r).astype(int) 

In [192]:
subs_agc = AgglomerativeClustering(n_clusters=2)
crisis_pa_adlt["subs_imp_med_agc_1"] = subs_agc.fit_predict(crisis_pa_adlt[[col + "_imp" for col in dmg_subs_cols_imp_1_med]])

In [200]:
subs_med_km = KMeans(n_clusters=10)
crisis_pa_adlt["subs_imp_med_km_1"] = subs_med_km.fit_predict(crisis_pa_adlt[[col + "_imp_freq" for col in dmg_subs_cols_imp_1_med]])

In [212]:
subs_sub_km = KMeans(n_clusters=2, max_iter=1000)
crisis_pa_adlt["subs_imp_sub_km_1"] = subs_sub_km.fit_predict(crisis_pa_adlt[[col + "_imp" for col in dmg_subs_cols_imp_1_sub]])

In [213]:
crisis_pa_adlt[["subs_imp_sub_km_1"] + ["mentalhealth"] + dmg_subs_cols_imp_1_sub].groupby(["subs_imp_sub_km_1"]).agg(["mean", "count"])

mentalhealth         alcohol       cigarettes        \
                          mean count      mean count       mean count   
subs_imp_sub_km_1                                                       
0                     2.662547  3965  1.436190  2194   1.293088  2228   
1                     2.699821  1116  5.268605  2029   2.307692  2028   

                       vape         tobacco       marijuana          heroin  \
                       mean count      mean count      mean count      mean   
subs_imp_sub_km_1                                                             
0                  1.302989  2208  1.034343  2213  1.226067  2225  1.004939   
1                  1.858346  2019  1.216633  2008  1.739001  2023  1.055720   

                          cocaine        
                  count      mean count  
subs_imp_sub_km_1                        
0                  2227  1.024898  2209  
1                  2028  1.128840  2018

In [166]:
crisis_pa_adlt[dmg_subs_cols_imp_1_med].corr(method="spearman")

,rxpain,rxsleeping,rxstimulants,rxother
rxpain,1.000000,0.213177,0.194715,0.249505
rxsleeping,0.213177,1.000000,0.304109,0.246675
rxstimulants,0.194715,0.304109,1.000000,0.276484
rxother,0.249505,0.246675,0.276484,1.000000


In [233]:
# Filter data for cases where mental health is available.
crisis_pa_adlt_mh = crisis_pa_adlt[~pandas.isna(crisis_pa_adlt["mentalhealth"])]
crisis_pa_adlt_mh["mentalhealth"] = crisis_pa_adlt_mh["mentalhealth"].astype(int)

C:\Users\sanji\AppData\Local\Temp/ipykernel_20596/2453401777.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crisis_pa_adlt_mh["mentalhealth"] = crisis_pa_adlt_mh["mentalhealth"].astype(int)


In [248]:
for v in crisis_pa_adlt_mh["mentalhealth"].unique():
    # Filter data for each mental health value.
    crisis_pa_adlt_mh_1 = crisis_pa_adlt_mh.copy(True)
    crisis_pa_adlt_mh_1["mh_tgt"] = crisis_pa_adlt_mh_1["mentalhealth"].apply(lambda r: 1 if r==v else 0)
    # Print information
    print("INFO: Training model for Mental Health - " + str(v))
    X_train, X_test, y_train, y_test = train_test_split(crisis_pa_adlt_mh_1[[col + "_imp" for col in dmg_subs_cols_imp_1]], crisis_pa_adlt_mh_1["mh_tgt"], test_size=0.2, random_state=100)
    # Train a Gradient boosted classifier
    gb_mh = GradientBoostingClassifier(learning_rate=0.05, n_estimators=500, max_depth=5)
    gb_mh = gb_mh.fit(X_train, y_train)
    # Predict on training and test
    y_pred_train = gb_mh.predict(X_train)
    y_pred_test = gb_mh.predict(X_test)
    # Compute confusion matrix for train and test
    cf_train = confusion_matrix(y_train, y_pred_train)
    cf_test = confusion_matrix(y_test, y_pred_test)
    # Test AUC
    auc_test = roc_auc_score(y_test, y_pred_test)
    # Print confusion matrix
    print("INFO: Training Metrics")
    print(cf_train)
    print("INFO: Testing metrics")
    print(cf_test)
    print(auc_test)
    print("")


INFO: Training model for Mental Health - 1
INFO: Training Metrics
[[3464    1]
 [ 576   23]]
INFO: Testing metrics
[[851   4]
 [162   0]]
0.4976608187134503

INFO: Training model for Mental Health - 2
INFO: Training Metrics
[[2823    7]
 [1140   94]]
INFO: Testing metrics
[[697  17]
 [291  12]]
0.507897218293258

INFO: Training model for Mental Health - 3
INFO: Training Metrics
[[2703   26]
 [1175  160]]
INFO: Testing metrics
[[673  18]
 [315  11]]
0.5038465636181226

INFO: Training model for Mental Health - 4
INFO: Training Metrics
[[3365    7]
 [ 581  111]]
INFO: Testing metrics
[[828  23]
 [162   4]]
0.4985346792575709

INFO: Training model for Mental Health - 5
INFO: Training Metrics
[[3860    0]
 [ 182   22]]
INFO: Testing metrics
[[954   3]
 [ 60   0]]
0.49843260188087773



In [251]:
gb_mh.feature_names_in_, gb_mh.feature_importances_

(array(['alcohol_imp', 'cigarettes_imp', 'vape_imp', 'tobacco_imp',
        'marijuana_imp', 'heroin_imp', 'cocaine_imp', 'rxpain_imp',
        'rxsleeping_imp', 'rxstimulants_imp', 'rxother_imp'], dtype=object),
 array([0.28464337, 0.16478001, 0.09523907, 0.07600106, 0.13323093,
        0.00416749, 0.0252226 , 0.11279175, 0.05514027, 0.02134166,
        0.0274418 ]))

In [275]:
# Imputation for age, tall and weight
crisis_pa_adlt["age_imp"] = crisis_pa_adlt["age"]
crisis_pa_adlt["tall_cm_imp"] = crisis_pa_adlt["tall_cm"]
crisis_pa_adlt["weight_kg_imp"] = crisis_pa_adlt["weight_kg"]
# Medians
age_median = crisis_pa_adlt["age"].median()
tall_cm_median = crisis_pa_adlt["tall_cm"].median()
weight_kg_median = crisis_pa_adlt["weight_kg"].median()
# Replace with median
crisis_pa_adlt["age_imp"] = crisis_pa_adlt["age"].apply(lambda r: age_median if numpy.isnan(r) else r)
crisis_pa_adlt["tall_cm_imp"] = crisis_pa_adlt["tall_cm"].apply(lambda r: tall_cm_median if numpy.isnan(r) else r)
crisis_pa_adlt["weight_kg_imp"] = crisis_pa_adlt["weight_kg"].apply(lambda r: weight_kg_median if numpy.isnan(r) else r)

In [276]:
# Gender dictionary for imputation
# 1, Man | 2, Woman | 3, Non-binary | 4, Transgender man | 5, Transgender woman | 6, Other
crisis_pa_adlt["gender_m"] = crisis_pa_adlt["gender"].apply(lambda r: 1 if r==1 else 0)
crisis_pa_adlt["gender_f"] = crisis_pa_adlt["gender"].apply(lambda r: 1 if r==2 else 0)
crisis_pa_adlt["gender_o"] = crisis_pa_adlt["gender"].apply(lambda r: 1 if r > 2 else 0)
# Unknown
crisis_pa_adlt["gender_u"] = crisis_pa_adlt["gender"].apply(lambda r: 1 if numpy.isnan(r) else 0)

In [279]:
crisis_pa_adlt["cigarettesyn_imp"] = crisis_pa_adlt["cigarettesyn"].apply(lambda r: 0 if numpy.isnan(r) else r)

In [283]:
crisis_pa_adlt["hoursofsleepweekdays_imp"] = crisis_pa_adlt["hoursofsleepweekdays"]
crisis_pa_adlt["hoursofsleepweekends_imp"] = crisis_pa_adlt["hoursofsleepweekends"]
hoswd_median = crisis_pa_adlt["hoursofsleepweekdays"].median()
hoswe_median = crisis_pa_adlt["hoursofsleepweekends"].median()
crisis_pa_adlt["hoursofsleepweekdays_imp"] = crisis_pa_adlt["hoursofsleepweekdays"].apply(lambda r: hoswd_median if numpy.isnan(r) else r)
crisis_pa_adlt["hoursofsleepweekends_imp"] = crisis_pa_adlt["hoursofsleepweekends"].apply(lambda r: hoswe_median if numpy.isnan(r) else r)

In [284]:
# Trouble sleeping - Imputation
crisis_pa_adlt["troublesleeping_imp"] = crisis_pa_adlt["troublesleeping"].apply(lambda r: 0 if numpy.isnan(r) or r <= 2 else 1)

In [288]:
crisis_pa_adlt["sleepquality_imp"] = crisis_pa_adlt["sleepquality"].apply(lambda r: 1 if numpy.isnan(r) or r <= 2 else 0)

In [296]:
# Imputing the health conditions columns
for k in range(1, 21):
    crisis_pa_adlt["healthconditions___" + str(k) + "_imp"] = crisis_pa_adlt["healthconditions___" + str(k)].apply(lambda r: 0 if numpy.isnan(r) else r)

In [302]:
dmg_hlth_cols_imp = [
    "age_imp",
    "gender_m",
    "gender_f",
    "gender_o",
    "gender_u",
    "tall_cm_imp",
    "weight_kg_imp",
    "cigarettesyn_imp",
    "healthconditions___1_imp",
    "healthconditions___2_imp",
    "healthconditions___3_imp",
    "healthconditions___4_imp",
    "healthconditions___5_imp",
    "healthconditions___6_imp",
    "healthconditions___7_imp",
    "healthconditions___8_imp",
    "healthconditions___9_imp",
    "healthconditions___10_imp",
    "healthconditions___11_imp",
    "healthconditions___12_imp",
    "healthconditions___13_imp",
    "healthconditions___14_imp",
    "healthconditions___15_imp",
    "healthconditions___16_imp",
    "healthconditions___17_imp",
    "healthconditions___18_imp",
    "healthconditions___19_imp",
    "healthconditions___20_imp",
    "hoursofsleepweekdays_imp",
    "hoursofsleepweekends_imp",
    "troublesleeping_imp",
    "sleepquality_imp"
]

In [300]:
# Filter data for cases where mental health is available.
crisis_pa_adlt_mh = crisis_pa_adlt[~pandas.isna(crisis_pa_adlt["mentalhealth"])]
crisis_pa_adlt_mh["mentalhealth"] = crisis_pa_adlt_mh["mentalhealth"].astype(int)

C:\Users\sanji\AppData\Local\Temp/ipykernel_20596/2453401777.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crisis_pa_adlt_mh["mentalhealth"] = crisis_pa_adlt_mh["mentalhealth"].astype(int)


In [303]:
for v in crisis_pa_adlt_mh["mentalhealth"].unique():
    # Filter data for each mental health value.
    crisis_pa_adlt_mh_1 = crisis_pa_adlt_mh.copy(True)
    crisis_pa_adlt_mh_1["mh_tgt"] = crisis_pa_adlt_mh_1["mentalhealth"].apply(lambda r: 1 if r==v else 0)
    # Print information
    print("INFO: Training model for Mental Health - " + str(v))
    X_train, X_test, y_train, y_test = train_test_split(crisis_pa_adlt_mh_1[dmg_hlth_cols_imp], crisis_pa_adlt_mh_1["mh_tgt"], test_size=0.2, random_state=100)
    # Train a Gradient boosted classifier
    gb_mh = GradientBoostingClassifier(learning_rate=0.05, n_estimators=500, max_depth=5)
    gb_mh = gb_mh.fit(X_train, y_train)
    # Predict on training and test
    y_pred_train = gb_mh.predict(X_train)
    y_pred_test = gb_mh.predict(X_test)
    # Compute confusion matrix for train and test
    cf_train = confusion_matrix(y_train, y_pred_train)
    cf_test = confusion_matrix(y_test, y_pred_test)
    # Test AUC
    auc_test = roc_auc_score(y_test, y_pred_test)
    # Print confusion matrix
    print("INFO: Training Metrics")
    print(cf_train)
    print("INFO: Testing metrics")
    print(cf_test)
    print(auc_test)
    print("")


INFO: Training model for Mental Health - 1
INFO: Training Metrics
[[3461    4]
 [ 251  348]]
INFO: Testing metrics
[[837  18]
 [141  21]]
0.5542884990253412

INFO: Training model for Mental Health - 2
INFO: Training Metrics
[[2800   30]
 [ 473  761]]
INFO: Testing metrics
[[631  83]
 [242  61]]
0.5425368167068807

INFO: Training model for Mental Health - 3
INFO: Training Metrics
[[2707   22]
 [ 473  862]]
INFO: Testing metrics
[[605  86]
 [264  62]]
0.53286337041542

INFO: Training model for Mental Health - 4
INFO: Training Metrics
[[3369    3]
 [ 202  490]]
INFO: Testing metrics
[[795  56]
 [135  31]]
0.5604710262908272

INFO: Training model for Mental Health - 5
INFO: Training Metrics
[[3859    1]
 [  21  183]]
INFO: Testing metrics
[[946  11]
 [ 52   8]]
0.5609195402298851



In [305]:
# Filter data for cases where mental health is available.
crisis_pa_adlt_ph = crisis_pa_adlt[~pandas.isna(crisis_pa_adlt["physicalhealth"])]
crisis_pa_adlt_ph["physicalhealth"] = crisis_pa_adlt_ph["physicalhealth"].astype(int)

C:\Users\sanji\AppData\Local\Temp/ipykernel_20596/147150225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crisis_pa_adlt_ph["physicalhealth"] = crisis_pa_adlt_ph["physicalhealth"].astype(int)


In [313]:
for v in crisis_pa_adlt_ph["physicalhealth"].unique():
    # Filter data for each mental health value.
    crisis_pa_adlt_ph_1 = crisis_pa_adlt_ph.copy(True)
    crisis_pa_adlt_ph_1["ph_tgt"] = crisis_pa_adlt_ph_1["physicalhealth"].apply(lambda r: 1 if r==v else 0)
    # Print information
    print("INFO: Training model for Physical Health - " + str(v))
    X_train, X_test, y_train, y_test = train_test_split(crisis_pa_adlt_ph_1[dmg_hlth_cols_imp], crisis_pa_adlt_ph_1["ph_tgt"], test_size=0.2, random_state=100)
    # Train a Gradient boosted classifier
    gb_mh = GradientBoostingClassifier(learning_rate=0.1, n_estimators=1000, max_depth=4)
    gb_mh = gb_mh.fit(X_train, y_train)
    # Predict on training and test
    y_pred_train = gb_mh.predict(X_train)
    y_pred_test = gb_mh.predict(X_test)
    # Compute confusion matrix for train and test
    cf_train = confusion_matrix(y_train, y_pred_train)
    cf_test = confusion_matrix(y_test, y_pred_test)
    # Test AUC
    auc_test = roc_auc_score(y_test, y_pred_test)
    # Print confusion matrix
    print("INFO: Training Metrics")
    print(cf_train)
    print("INFO: Testing metrics")
    print(cf_test)
    print(auc_test)
    print("")


INFO: Training model for Physical Health - 2
INFO: Training Metrics
[[2575   62]
 [ 200 1227]]
INFO: Testing metrics
[[527 131]
 [234 125]]
0.5745506345725631

INFO: Training model for Physical Health - 3
INFO: Training Metrics
[[2531   45]
 [ 229 1259]]
INFO: Testing metrics
[[521 148]
 [235 113]]
0.5517434668316066

INFO: Training model for Physical Health - 1
INFO: Training Metrics
[[3586    3]
 [  48  427]]
INFO: Testing metrics
[[848  36]
 [114  19]]
0.5510665804783451

INFO: Training model for Physical Health - 4
INFO: Training Metrics
[[3506    0]
 [  45  513]]
INFO: Testing metrics
[[826  43]
 [112  36]]
0.5968805399185145

INFO: Training model for Physical Health - 5
INFO: Training Metrics
[[3948    0]
 [   0  116]]
INFO: Testing metrics
[[980   8]
 [ 28   1]]
0.5131927963143935



In [310]:
gb_mh.feature_importances_, gb_mh.feature_names_in_

(array([0.18890326, 0.00525318, 0.00436457, 0.00666656, 0.01604078,
        0.15479752, 0.2005008 , 0.0125825 , 0.02701068, 0.02808149,
        0.00496811, 0.0081352 , 0.00378124, 0.0101784 , 0.00675944,
        0.01229843, 0.01449268, 0.00143525, 0.01194944, 0.01888685,
        0.09036721, 0.00705035, 0.00171739, 0.00792538, 0.00723214,
        0.00406783, 0.        , 0.        , 0.0651591 , 0.0479111 ,
        0.00477314, 0.02670999]),
 array(['age_imp', 'gender_m', 'gender_f', 'gender_o', 'gender_u',
        'tall_cm_imp', 'weight_kg_imp', 'cigarettesyn_imp',
        'healthconditions___1_imp', 'healthconditions___2_imp',
        'healthconditions___3_imp', 'healthconditions___4_imp',
        'healthconditions___5_imp', 'healthconditions___6_imp',
        'healthconditions___7_imp', 'healthconditions___8_imp',
        'healthconditions___9_imp', 'healthconditions___10_imp',
        'healthconditions___11_imp', 'healthconditions___12_imp',
        'healthconditions___13_imp', 'health

In [318]:
for v in crisis_pa_adlt_ph["physicalhealth"].unique():
    # Filter data for each mental health value.
    crisis_pa_adlt_ph_1 = crisis_pa_adlt_ph.copy(True)
    crisis_pa_adlt_ph_1["mh_tgt"] = crisis_pa_adlt_ph_1["physicalhealth"].apply(lambda r: 1 if r==v else 0)
    # Print information
    print("INFO: Training model for Physical Health - " + str(v))
    X_train, X_test, y_train, y_test = train_test_split(crisis_pa_adlt_ph_1[[col + "_imp_freq" for col in dmg_subs_cols_imp_1]], crisis_pa_adlt_ph_1["mh_tgt"], test_size=0.2, random_state=100)
    # Train a Gradient boosted classifier
    gb_mh = GradientBoostingClassifier(learning_rate=0.05, n_estimators=500, max_depth=5)
    gb_mh = gb_mh.fit(X_train, y_train)
    # Predict on training and test
    y_pred_train = gb_mh.predict(X_train)
    y_pred_test = gb_mh.predict(X_test)
    # Compute confusion matrix for train and test
    cf_train = confusion_matrix(y_train, y_pred_train)
    cf_test = confusion_matrix(y_test, y_pred_test)
    # Test AUC
    auc_test = roc_auc_score(y_test, y_pred_test)
    # Print confusion matrix
    print("INFO: Training Metrics")
    print(cf_train)
    print("INFO: Testing metrics")
    print(cf_test)
    print(auc_test)
    print("")


INFO: Training model for Physical Health - 2
INFO: Training Metrics
[[2620   17]
 [1282  145]]
INFO: Testing metrics
[[635  23]
 [344  15]]
0.5034141612550906

INFO: Training model for Physical Health - 3
INFO: Training Metrics
[[2555   21]
 [1310  178]]
INFO: Testing metrics
[[641  28]
 [329  19]]
0.5063720942219473

INFO: Training model for Physical Health - 1
INFO: Training Metrics
[[3588    1]
 [ 450   25]]
INFO: Testing metrics
[[882   2]
 [133   0]]
0.498868778280543

INFO: Training model for Physical Health - 4
INFO: Training Metrics
[[3500    6]
 [ 491   67]]
INFO: Testing metrics
[[860   9]
 [144   4]]
0.5083351475756539

INFO: Training model for Physical Health - 5
INFO: Training Metrics
[[3947    1]
 [ 101   15]]
INFO: Testing metrics
[[983   5]
 [ 28   1]]
0.5147110149378753

